In [ ]:
from pathlib import Path
project_root = Path.cwd().parent
data_dir = project_root / "data" / "raw" / "fer2013"

Looking in: /Users/mannvaswani/Desktop/DeceptiVision/data/raw/fer2013
Train exists: True
Test exists: True


In [19]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
import matplotlib.pyplot as plt
import torch.optim as optim

device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
device



device(type='mps')

In [16]:

transform = transforms.Compose([
    transforms.Grayscale(),
    transforms.Resize((48, 48)),
    transforms.ToTensor()
])


train_data = datasets.ImageFolder(data_dir / "train", transform=transform)
test_data  = datasets.ImageFolder(data_dir / "test", transform=transform)




In [17]:
class EmotionCNN(nn.Module):
    def __init__(self):
        super(EmotionCNN, self).__init__()
        
        self.conv_layers = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 6 * 6, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 7)   # 7 emotions in FER2013
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.fc_layers(x)
        return x

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)


In [20]:
model = EmotionCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005)


In [21]:
epochs = 10

for epoch in range(epochs):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{epochs}]  Loss: {running_loss/len(train_loader):.4f}")


Epoch [1/10]  Loss: 1.6649
Epoch [2/10]  Loss: 1.4601
Epoch [3/10]  Loss: 1.3480
Epoch [4/10]  Loss: 1.2601
Epoch [5/10]  Loss: 1.1863
Epoch [6/10]  Loss: 1.1230
Epoch [7/10]  Loss: 1.0598
Epoch [8/10]  Loss: 1.0001
Epoch [9/10]  Loss: 0.9365
Epoch [10/10]  Loss: 0.8686


In [22]:
correct = 0
total = 0
model.eval()

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print("Test Accuracy:", 100 * correct / total, "%")


Test Accuracy: 57.66230147673447 %
